# API for Game Data (Not Needed)

In [55]:
import requests


def get_season_data(year: int):

    url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}"
    headers = {"User-Agent": "xcs5hg@virginia.edu"}

    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json()

season_data = get_season_data(2024)

season_types = season_data["types"]["items"]
reg_season = next(t for t in season_types if t["name"] == "Regular Season")

reg_season


{'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2?lang=en&region=us',
 'id': '2',
 'type': 2,
 'name': 'Regular Season',
 'abbreviation': 'reg',
 'year': 2024,
 'startDate': '2024-09-05T07:00Z',
 'endDate': '2025-01-08T07:59Z',
 'hasGroups': False,
 'hasStandings': True,
 'hasLegs': False,
 'groups': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2/groups?lang=en&region=us'},
 'weeks': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2/weeks?lang=en&region=us'},
 'corrections': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2/corrections?lang=en&region=us'},
 'leaders': {'$ref': 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2024/types/2/leaders?lang=en&region=us'},
 'slug': 'regular-season'}

In [74]:
all_game_urls = all_game_urls[:3]

In [75]:
import pandas as pd

games_data = []

for game_url in all_game_urls:
    game = get_json(game_url)
    
    comp = game["competitions"][0]
    date = comp.get("date")
    
    competitors = comp["competitors"]
    home_comp = next(c for c in competitors if c.get("homeAway") == "home")
    away_comp = next(c for c in competitors if c.get("homeAway") == "away")
    
    home_team_url = home_comp["team"]["$ref"]
    away_team_url = away_comp["team"]["$ref"]
    
    home_team_data = get_json(home_team_url)
    away_team_data = get_json(away_team_url)
    
    games_data.append({
        "game_id": game.get("id"),
        "date": date,
        "home_team": home_team_data.get("displayName") or home_team_data.get("shortDisplayName"),
        "home_abbr": home_team_data.get("abbreviation"),
        "away_team": away_team_data.get("displayName") or away_team_data.get("shortDisplayName"),
        "away_abbr": away_team_data.get("abbreviation"),
        "status": comp.get("status", {}).get("type", {}).get("name")
    })

df_schedule = pd.DataFrame(games_data)
print(df_schedule.head())

Empty DataFrame
Columns: []
Index: []


# Odds API

In [56]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_game_odds_and_probabilities(game_id: int):

    base = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl"
    
    def get_json(url):
        r = requests.get(url, headers=header)
        r.raise_for_status()
        return r.json()

    event_url = f"{base}/events/{game_id}"
    event = get_json(event_url)

    comp = event["competitions"][0]
    comp_id = comp["id"]


    probs_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/probabilities?limit=500"
    )
    probabilities = get_json(probs_url)

    odds_url = (
        f"{base}/events/{game_id}/competitions/{comp_id}/odds"
    )
    odds = get_json(odds_url)

    return {
        "game_id": game_id,
        "competition_id": comp_id,
        "event": event,
        "probabilities": probabilities,
        "odds": odds,
    }

In [59]:
odds = get_game_odds_and_probabilities(401671789)

In [71]:
pd.json_normalize(odds['odds']['items'])

,$ref,details,overUnder,spread,overOdds,underOdds,links,moneylineWinner,spreadWinner,provider.$ref,...,bettingOdds.teamOdds.preMatchSpreadAway.betSlipUrl,bettingOdds.teamOdds.preMatchTotalUnder.oddId,bettingOdds.teamOdds.preMatchTotalUnder.value,bettingOdds.teamOdds.preMatchTotalUnder.betSlipUrl,bettingOdds.teamOdds.preMatchTotalHandicap.oddId,bettingOdds.teamOdds.preMatchTotalHandicap.value,bettingOdds.teamOdds.preMatchTotalHandicap.betSlipUrl,bettingOdds.teamOdds.preMatchSpreadHandicapHome.oddId,bettingOdds.teamOdds.preMatchSpreadHandicapHome.value,bettingOdds.teamOdds.preMatchSpreadHandicapHome.betSlipUrl
0,http://sports.core.api.espn.com/v2/sports/foot...,KC -2.5,45.5,-2.5,-115.0,-105.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://sports.core.api.espn.com/v2/sports/foot...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,10/11,https://www.bet365.com/dl/sportsbookredirect?a...,484455341,46.5,https://www.bet365.com/dl/sportsbookredirect?a...,484455319,-2.5,https://www.bet365.com/dl/sportsbookredirect?a...
2,http://sports.core.api.espn.com/v2/sports/foot...,KC -10.5,48.5,-10.5,185.0,-275.0,"[{'language': 'en-US', 'rel': ['home', 'deskto...",False,False,http://sports.core.api.espn.com/v2/sports/foot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import requests

header = {"User-Agent": "xcs5hg@virginia.edu"}

def get_scoreboard(start_date: str, end_date: str | None = None, limit: int = 1000):

    base_url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
    header = {"User-Agent": "xcs5hg@virginia.edu"}


    # Build dates parameter: either 'YYYYMMDD' or 'YYYYMMDD-YYYYMMDD'
    if end_date is not None:
        dates_param = f"{start_date}-{end_date}"
    else:
        dates_param = start_date  # can be 'YYYY', 'YYYYMMDD', etc.

    params = {
        "limit": limit,
        "dates": dates_param,
        # optional cache-buster if you want:
        # "_": "1577413600",
    }

    r = requests.get(base_url, headers=header, params=params)
    r.raise_for_status()
    return r.json()

# Play by play API 

In [52]:
import requests

def get_game_plays(game_id: int):

    header = {"User-Agent": "xcs5hg@virginia.edu"}

    event_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}"
    )
    event = requests.get(event_url, headers=header).json()

    competition = event["competitions"][0]
    comp_id = competition["id"]

    plays_url = (
        f"https://sports.core.api.espn.com/v2/"
        f"sports/football/leagues/nfl/events/{game_id}/competitions/{comp_id}/plays?limit=500"
    )

    plays = requests.get(plays_url, headers=header).json()
    
    record = []
    
    for item in plays["items"]:
        record.append(item)
        
    keep_cols = [
    "text",
    "period.number",
    "clock.value",
    "start.down",
    "start.distance",
    "start.yardLine",
    "start.yardsToEndzone",
    "end.yardsToEndzone",
    "homeScore",
    "awayScore",
    "statYardage",
    "scoringPlay",
    "scoreValue",
    "type.text",
    "type.abbreviation"]
    
    data = pd.json_normalize(record)
    data = data[keep_cols]  
    
    
    return data

In [53]:
plays = get_game_plays(401671789)
plays


,text,period.number,clock.value,start.down,start.distance,start.yardLine,start.yardsToEndzone,end.yardsToEndzone,homeScore,awayScore,statYardage,scoringPlay,scoreValue,type.text,type.abbreviation
0,GAME,0,900.0,0,0,0,0,65,0,0,0,False,0,Coin Toss,NaN
1,H.Butker kicks 65 yards from KC 35 to end zone...,1,900.0,0,0,35,65,70,0,0,0,False,0,Kickoff,K
2,(Shotgun) D.Henry left end to BLT 32 for 2 yar...,1,900.0,1,10,70,70,68,0,0,2,False,0,Rush,RUSH
3,(Shotgun) L.Jackson pass short right to Z.Flow...,1,859.0,2,8,68,68,73,0,0,-5,False,0,Pass Reception,REC
4,(Shotgun) L.Jackson pass short right to J.Hill...,1,835.0,2,13,73,73,71,0,0,2,False,0,Pass Reception,REC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,"(No Huddle, Shotgun) L.Jackson pass incomplete...",4,18.0,1,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
185,Timeout #4 by BLT at 00:10.,4,10.0,2,10,10,10,10,27,20,0,False,0,Timeout,TO
186,(Shotgun) L.Jackson pass incomplete short midd...,4,10.0,2,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN
187,(Shotgun) L.Jackson pass short middle to I.Lik...,4,5.0,3,10,10,10,10,27,20,0,False,0,Pass Incompletion,NaN


In [54]:
len(plays)

189